A general testing module that uses unit testing

In [1]:
import civiActivityReport_selectBestTrans as best
import os
import sys
import unittest
import pandas as pd
import inspect

In [ ]:
#list all of the names inside module best
dir(best)

In [2]:
os.chdir('/home/mofongo/Documents/ghfc/membershipReportsCIVI/membershipReportingLogicSampleReports')


In [3]:

activityReport = pd.read_csv('./selectActivityReport_10072024.csv')
#activityReport.columns = [i.replace(' ','_')+'_act' for i in list(activityReport.columns)]
activityReport.columns = [i.replace(' ','_')+'_act' for i in list(activityReport.columns)]

In [ ]:
activityReport.columns

In [4]:

activityReport = activityReport.map(lambda x: x.strip() if isinstance(x,str) else x)
activityReport = activityReport.assign(Activity_Date_DT_act = pd.to_datetime(activityReport['Activity_Date_act'], format = '%Y-%m-%d %H:%M'))
activityReport.drop_duplicates(inplace=True, ignore_index = True, subset = ['Activity_Type_act', 'Subject_act', 'Activity_Date_act', 'Activity_Status_act', 'Activity_Date_DT_act','Target_Email_act'])

In [5]:
#write out all the functions that return a boolean, which I used to test truthiness on my TestBest array of assertTrue functions
df = pd.DataFrame()
isinstance(df,pd.core.frame.DataFrame)

True

In [ ]:
activityReport.columns

In [ ]:
class TestBest(unittest.TestCase):
    
    def test_intake_file(self):
        df = best.intake_file(activityReport)
        self.assertTrue(isinstance(df,pd.core.frame.DataFrame),'result of intake_file is not a df')

        self.assertTrue(len(df) > 0,'df from intake_file is not adequate size')

    def test_apply_rules(self):
        df = best.intake_file(activityReport)
        ser = best.apply_rules(df)
        #len(ser) > 0
        #df.shape[0] > len(ser)
        self.assertTrue(len(ser) > 0,f'the DataFrame is not longer than Series')

    def test_apply_exclusion(self):
        df_group = best.intake_file(activityReport)
        ser = best.apply_rules(df_group)
        #function signature: apply_exclusion(inclusion_ser,df_grouped,activityReport)
        result_df = best.apply_exclusion(ser,df_group,activityReport)
        self.assertTrue(activityReport.shape[0] > result_df.shape[0])
    
    def test_remove_controller(self):
        best.remove_controller(activityReport)
        

unittest.main(argv=[''], verbosity=2, exit=False)

test_apply_exclusion (__main__.TestBest.test_apply_exclusion) ... ok
test_apply_rules (__main__.TestBest.test_apply_rules) ... ok
test_intake_file (__main__.TestBest.test_intake_file) ... ok

----------------------------------------------------------------------
Ran 3 tests in 1.612s

OK


In [9]:
print(inspect.getsource(best.apply_exclusion))

def apply_exclusion(inclusion_ser,df_grouped,activityReport):
    #inclusion_df = concat_grouped_ser from apply_rules()
    #exclusion_list = a list - all indices of df_grouped OUTSIDE of those we want to keep (ie contact_grouped_df <- the cumulative DF)

    #check that all expected columns exist
    if ('index' in df_grouped.columns) and ('index' in inclusion_ser.columns):
        exclusion_list = df_grouped.loc[~df_grouped['index'].isin(inclusion_ser['index']),'index'].to_list()
        #isolate the records to remove by negative indexing on concat_grouped_df

        #TODO inverse index activityReport by selecting indices NOT IN exclusion_list
        #inspection
        #df_grouped.loc[~df_grouped['index'].isin(concat_grouped_df['index']),:].sort_values(['Target_Email_act','Activity_Date_DT_act']).to_csv('/home/mofongo/Documents/ghfc/membershipReportsCIVI/records_to_remove.csv', index = False)
        #test = activityReport.loc[~df_grouped.loc[~df_grouped['index'].isin(concat_group

In [10]:
print(inspect.getsource(best.intake_file))

def intake_file(activityReport):

    #I removed some data munging here, as this should be done in another module or the calling ipynb; munging removed denoted w/ "*"

    #activityReport.columns = [i.replace(' ','_')+'_act' for i in list(activityReport.columns)]*

    #trim all string fields
    #strip all whitespace from each cell
    #activityReport = activityReport.map(lambda x: x.strip() if isinstance(x,str) else x) *

    #NOTE: Activity_Date_act field DOES NOT provide seconds
    #activityReport = activityReport.assign(Activity_Date_DT_act = pd.to_datetime(activityReport['Activity_Date_act'], format = '%Y-%m-%d %H:%M')) *

    try:
        activityReport = activityReport.drop_duplicates(ignore_index = True, subset = ['Activity_Type_act', 'Subject_act', 'Activity_Date_act', 'Activity_Status_act', 'Activity_Date_DT_act','Target_Email_act'])

        # ### SELECT BEST TRANSACTION LOGIC
        #NOTE: real dupes have multiple entries on fields: 'Target_Email_act','Activity_Date_DT_a